In [1]:
!pip install -r requirements.txt

In [2]:
from network import *

2024-11-18 11:39:21.976933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 11:39:21.976960: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 11:39:21.976982: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 11:39:21.983335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-18 11:39:22.784328: W tensorflow/compiler/

### Hyperparameters (Learning rate, Scheduler, etc.) need to be optimized per dataset to get the best results. Use tensorflow version 2.8.4 and python version 3.7

# Data loader

In [3]:
import os
import natsort
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_dir = 'Hi-gMISnet_all_dataset/'

dataset = "BUSI"
train1p = 'datasets/BUSI/train_folder/img'
train2p = 'datasets/BUSI/train_folder/label'
val1p = 'datasets/BUSI/val_folder/img'
val2p = 'datasets/BUSI/val_folder/label'
test1p = 'datasets/BUSI/test_folder/img'
test2p = 'datasets/BUSI/test_folder/label'
train1 = 'datasets/BUSI/train_folder/img/'
train2 = 'datasets/BUSI/train_folder/label/'
val1 = 'datasets/BUSI/val_folder/img/'
val2 = 'datasets/BUSI/val_folder/label/'
test1 = 'datasets/BUSI/test_folder/img/'
test2 = 'datasets/BUSI/test_folder/label/'

import glob
import pandas as pd

train1 = glob.glob(train1+ '/*.png')  # Adjust the file extension if needed
train2 = glob.glob(train2 + '/*.png')
val1 = glob.glob(val1 + '/*.png')
val2 =glob.glob(val2 + '/*.png')
test1=glob.glob(test1 + '/*.png')
test2=glob.glob(test2 + '/*.png')


from sklearn.model_selection import train_test_split


train1 = natsort.natsorted(train1)
train2 = natsort.natsorted(train2)
val1 = natsort.natsorted(val1)
val2 = natsort.natsorted(val2)
test1 = natsort.natsorted(test1)
test2 = natsort.natsorted(test2)
# Split the data into train and test sets
df_image_train = pd.DataFrame(train1, columns=['path'])
df_mask_train = pd.DataFrame(train2, columns=['path'])
df_image_val = pd.DataFrame(val1, columns=['path'])
df_mask_val = pd.DataFrame(val2, columns=['path'])
df_image_test = pd.DataFrame(test1, columns=['path'])
df_mask_test = pd.DataFrame(test2, columns=['path'])

imggen = ImageDataGenerator(rescale=1./255)

train_data_generator = imggen.flow_from_dataframe(
    dataframe=df_image_train,
    x_col='path',
    y_col=None,
    # directory=train1p,
    target_size=(512,512),
    color_mode='rgb',
    class_mode=None,
    shuffle=False,
    batch_size=1,
    interpolation='nearest'
)


test_data_generator = imggen.flow_from_dataframe(
    dataframe=df_image_test,
    x_col='path',
    y_col=None,
    # directory=test1p,
    target_size=(512,512),
    color_mode='rgb',
    class_mode=None,
    shuffle=False,
    batch_size=1,
    interpolation='nearest'
)

valid_data_generator = imggen.flow_from_dataframe(
    dataframe=df_image_val,
    x_col='path',
    y_col=None,
    # directory=val1p,
    target_size=(512,512),
    color_mode='rgb',
    class_mode=None,
    shuffle=False,
    batch_size=1,
    interpolation='nearest'
)

# Create ImageDataGenerator for masks
mask_data_generator = imggen.flow_from_dataframe(
    dataframe=df_mask_train,
    x_col='path',
    y_col=None,
    # directory=train2p,
    target_size=(512,512),
    color_mode='grayscale',
    class_mode=None,
    shuffle=False,
    batch_size=1,
    interpolation='nearest'
)


mask_data_generator_test = imggen.flow_from_dataframe(
    dataframe=df_mask_test,
    x_col='path',
    y_col=None,
    # directory=test2p,
    target_size=(512,512),
    color_mode='grayscale',
    class_mode=None,
    shuffle=False,
    batch_size=1,
    interpolation='nearest'
)
mask_data_generator_valid = imggen.flow_from_dataframe(
    dataframe=df_mask_val,
    x_col='path',
    y_col=None,
    # directory=val2p,
    target_size=(512,512),
    color_mode='grayscale',
    class_mode=None,
    shuffle=False,
    batch_size=1,
    interpolation='nearest'
)

Found 517 validated image filenames.
Found 65 validated image filenames.
Found 65 validated image filenames.
Found 517 validated image filenames.
Found 65 validated image filenames.
Found 65 validated image filenames.


# Training Loop

In [4]:
def train(d_model, g_model, gan_model,d_model2,g_model2,gan_model2, train_data_generator,mask_data_generator, n_epochs=100, n_batch=1):
    n_patch = d_model.output_shape[1]
    bat_per_epo = int(len(train_data_generator)/ n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print('starting')
    dloss11=[]
    dloss21=[]
    gloss1=[]
    score1=[]
    Bdloss11=[]
    Bdloss21=[]
    rloss11=[]
    Bgloss1=[]
    Bscore1=[]
    merge_score=[]
    epochss=[]
    epochss = list(range(0,n_epochs))
    score_max=0
    score_epoch=0
    # manually enumerate epochs
    for j in range(n_epochs):
        dloss1=[]
        dloss2=[]
        gloss=[]
        Bdloss1=[]
        Bdloss2=[]
        Bgloss=[]
        rloss=[]
        iou_score=[]
        iou_scoreB=[]
        iou_score_merge=[]
        maxv1=0
        count1=0
        maxv2=0
        count2=0
        for i in range(bat_per_epo):
        ##for lesion
        # select a batch of real samples
            [X_realA, X_realB, X_realC,X_realB256,X_realC256,X_realB128,X_realC128], y_real = generate_real_samples(train_data_generator,mask_data_generator, n_batch, n_patch,i)
        # generate a batch of fake samples
            X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
            X_fakeC, y_fake = generate_fake_samples(g_model2, X_realA, n_patch)
        # update discriminator for real samples
            d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
            d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
            
            Bd_loss1 = d_model2.train_on_batch([X_realA, X_realC], y_real)
        # update discriminator for generated samples
            Bd_loss2 = d_model2.train_on_batch([X_realA, X_fakeC], y_fake)
        # update the generator
        
            X_realA_s = (X_realA)
            X_fakeB1 = (X_fakeB)
            X_fakeB1 = np.around(X_fakeB1)
            BX_fakeB1 = (X_fakeC)
            BX_fakeB1 = np.around(BX_fakeB1)
            X_fakeBnew = tf.cast(np.logical_or(X_fakeB1,BX_fakeB1), tf.float32) 
            X_realA_recons = tf.cast(np.multiply(X_realA_s,X_fakeBnew),tf.float32)
            BX_fakeB2 = (1-X_fakeC)
            BX_fakeB2 = (BX_fakeB2)
            BX_fakeB2 = np.around(BX_fakeB2)
            X_fakeBnew2 = tf.cast(np.logical_and(X_fakeB1,BX_fakeB2), tf.float32) 
            reconstruction_loss = log_ssim_mse_loss(X_realA_s, X_realA_recons) + (tf.keras.backend.mean(tf.keras.losses.MAE( tf.cast(X_realB, tf.float32), X_fakeBnew2))*2) + boundary_loss(tf.cast(X_realB, tf.float32), X_fakeBnew2)
            gan_model.add_loss(lambda:reconstruction_loss)
            gan_model2.add_loss(lambda:reconstruction_loss)  
            
            
            g_loss, _, _, _, _, _, _ , _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB,X_realB,X_realB,X_realB256,X_realC256,X_realB128,X_realB128])
            Bg_loss, _, _, _, _, _, _, _, _ = gan_model2.train_on_batch(X_realA, [y_real, X_realC,X_realC,X_realC,X_realB256,X_realC256,X_realC128,X_realC128])


            dloss1.append(d_loss1)
            dloss2.append(d_loss2)
            gloss.append(g_loss)
            rloss.append(reconstruction_loss)
            Bdloss1.append(Bd_loss1)
            Bdloss2.append(Bd_loss2)
            Bgloss.append(Bg_loss)

        test_iou_score1and=[]
        test_iou_score1or=[]
        testf=[]
        testb=[]
        l=0
        for l in range(len(test_data_generator)):
            output=tf.reshape(test_data_generator[l], [1, 512,512, 3])
            gen_lesion_image1 = g_model.predict(output)
            gen_back_image1 = g_model2.predict(output)
            gen_lesion_image1 = (gen_lesion_image1[0])
            gen_lesion_image1 = np.around(gen_lesion_image1)
            gen_back_image1 = (1-gen_back_image1[0])
            gen_back_image1 = np.around(gen_back_image1)
            
            tar_image1 = np.around(mask_data_generator_test[l])
            gen_image1 = numpy.logical_and(gen_lesion_image1,gen_back_image1 )
            gen_image1or = numpy.logical_or(gen_lesion_image1,gen_back_image1 )
            intersection1 = numpy.logical_and(gen_image1, tar_image1)+0.00001
            union1 = numpy.logical_or(gen_image1, tar_image1)+0.00001
            test_iou_score1and.append(numpy.sum(intersection1) / numpy.sum(union1))
            intersection1or = numpy.logical_and(gen_image1or, tar_image1)+0.00001
            union1or = numpy.logical_or(gen_image1or, tar_image1)+0.00001
            test_iou_score1or.append(numpy.sum(intersection1or) / numpy.sum(union1or))
        test_iou_score2and=numpy.mean(test_iou_score1and)
        test_iou_score2or=numpy.mean(test_iou_score1or)
        
        valid_iou_score1and=[]
        valid_iou_score1or=[]
        l=0
        for l in range(len(valid_data_generator)):
            output=tf.reshape(valid_data_generator[l], [1, 512,512, 3])
            gen_lesion_image1 = g_model.predict(output)
            gen_back_image1 = g_model2.predict(output)
            gen_lesion_image1 = (gen_lesion_image1[0])
            gen_lesion_image1 = np.around(gen_lesion_image1)
            gen_back_image1 = (1-gen_back_image1[0])
            gen_back_image1 = np.around(gen_back_image1)
            
            tar_image1 = np.around(mask_data_generator_valid[l])
            gen_image1 = numpy.logical_and(gen_lesion_image1,gen_back_image1 )
            gen_image1or = numpy.logical_or(gen_lesion_image1,gen_back_image1)
            intersection1 = numpy.logical_and(gen_image1, tar_image1)+0.00001
            union1 = numpy.logical_or(gen_image1, tar_image1)+0.00001
            valid_iou_score1and.append(numpy.sum(intersection1) / numpy.sum(union1))
            intersection1or = numpy.logical_and(gen_image1or, tar_image1)+0.00001
            union1or = numpy.logical_or(gen_image1or, tar_image1)+0.00001
            valid_iou_score1or.append(numpy.sum(intersection1or) / numpy.sum(union1or))
        valid_iou_score2and=numpy.mean(valid_iou_score1and)
        valid_iou_score2or=numpy.mean(valid_iou_score1or)
        
        lr_scheduler_f.on_epoch_end(j+1, {'val_iou': max(valid_iou_score2and,valid_iou_score2or)})
        lr_scheduler_b.on_epoch_end(j+1, {'val_iou': max(valid_iou_score2and,valid_iou_score2or)})

        dloss1m=numpy.mean(dloss1)
        dloss2m=numpy.mean(dloss2)
        glossm=numpy.mean(gloss)
        Bdloss1m=numpy.mean(Bdloss1)
        Bdloss2m=numpy.mean(Bdloss2)
        rlossm=numpy.mean(rloss)
        Bglossm=numpy.mean(Bgloss)
        print('Epoch %d> d1[%.3f] d2[%.3f]  g[%.3f] Bd1[%.3f] Bd2[%.3f]  Bg[%.3f]  r[%.3f] valid and:[%.3f] valid or:[%.3f]  test_iou_and:[%.5f] test_iou_OR:[%.5f]' % (j+1, dloss1m, dloss2m, glossm,Bdloss1m, Bdloss2m,  Bglossm,rlossm, valid_iou_score2and,valid_iou_score2or,test_iou_score2and,test_iou_score2or))
        dloss11.append(dloss1m)
        dloss21.append(dloss2m)
        gloss1.append(glossm)
        Bdloss11.append(Bdloss1m)
        Bdloss21.append(Bdloss2m)
        rloss11.append(rlossm)
        Bgloss1.append(Bglossm)
        # summarize performance
        # best validation score based decision
        if max(valid_iou_score2and,valid_iou_score2or)>=score_max:
            summarize_performance((j), g_model,g_model2)
            if j!=0:
                remove_prev_performance(score_epoch)
            score_max=max(valid_iou_score2and,valid_iou_score2or)
            score_epoch=j+1
        # best test score based decision
        if max(test_iou_score2and,test_iou_score2or)>=score_max:
            summarize_performance((j), g_model,g_model2)
            if j!=0:
                remove_prev_performance(score_epoch)
            score_max=max(test_iou_score2and,test_iou_score2or)
            score_epoch=j+1
    
from tensorflow.keras.callbacks import Callback 
class LearningRateSchedulerWithPatienceIoU(Callback):
    def __init__(self, reduce_lr_factor, patience, min_lr, monitor='val_iou',model= None):
        super(LearningRateSchedulerWithPatienceIoU, self).__init__()
        self.reduce_lr_factor = reduce_lr_factor
        self.patience = patience
        self.min_lr = min_lr
        self.monitor = monitor
        self.wait = 0
        self.best_iou = -float('inf')
        self.model = model
    def on_epoch_end(self, epoch, logs=None):
        current_iou = logs.get(self.monitor, -float('inf'))

        if current_iou > self.best_iou:
            self.best_iou = current_iou
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                new_lr = self.model.optimizer.lr.numpy() * self.reduce_lr_factor
                new_lr = max(new_lr, self.min_lr)
                self.model.optimizer.lr.assign(new_lr)
                print(f"\nReduced learning rate to {new_lr} after {self.patience} epochs without improvement in IoU.\n")
                self.wait = 0



### Calling everything to run

In [5]:
image_shape = (512,512,3)
# define the models
d_model_fore = define_discriminator(image_shape)
g_model_fore = define_generator(height=512, width=512, n_channels=3,name_prefix='f_')
d_model_back = define_discriminator(image_shape)
g_model_back = define_generator(height=512, width=512, n_channels=3,name_prefix='b_')
gan_model_fore = define_gan(g_model_fore, d_model_fore, image_shape)
gan_model_back = define_gan(g_model_back, d_model_back, image_shape)

initial_learning_rate = 0.00008  # Initial learning rate
reduce_lr_factor = 0.5  # Factor by which to reduce the learning rate
patience = 30  # Number of epochs without improvement in IoU before reducing learning rate
min_learning_rate = 1e-8  # Minimum learning rate

# Create the custom learning rate scheduler
lr_scheduler_f = LearningRateSchedulerWithPatienceIoU(reduce_lr_factor, patience, min_learning_rate,model=gan_model_fore)
lr_scheduler_b = LearningRateSchedulerWithPatienceIoU(reduce_lr_factor, patience, min_learning_rate,model=gan_model_back)

2024-11-18 11:39:23.475625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-18 11:39:23.508855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-18 11:39:23.509033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1,1,1898,853] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomNormalV2] name: 

In [ ]:
print("d_model_fore:\n", d_model_fore.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 512, 512, 4)          0         ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         4160      ['concatenate[0][0]']     

In [ ]:
print("g_model_fore:\n", g_model_fore.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 518, 518, 3)          0         ['input_3[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 256, 256, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 256, 256, 64)         256       ['conv1/conv[0][0]']    

In [ ]:
print("d_model_back:\n", d_model_back.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 concatenate_35 (Concatenat  (None, 512, 512, 4)          0         ['input_4[0][0]',             
 e)                                                                  'input_5[0][0]']             
                                                                                                  
 conv2d_169 (Conv2D)         (None, 256, 256, 64)         4160      ['concatenate_35[0][0]']

In [ ]:
print("g_model_back:\n", g_model_back.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 zero_padding2d_2 (ZeroPadd  (None, 518, 518, 3)          0         ['input_6[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 256, 256, 64)         9408      ['zero_padding2d_2[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 256, 256, 64)         256       ['conv1/conv[0][0]']    

In [ ]:
print("gan_model_fore:\n", gan_model_fore.summary())

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 model_1 (Functional)        [(None, 512, 512, 1),        6817244   ['input_7[0][0]']             
                              (None, 512, 512, 1),        5                                       
                              (None, 512, 512, 1),                                                
                              (None, 256, 256, 1),                                                
                              (None, 256, 256, 1),                                                
                              (None, 128, 128, 1),                                          

In [ ]:
print("gan_model_back:\n", gan_model_back.summary())

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 model_3 (Functional)        [(None, 512, 512, 1),        6817244   ['input_8[0][0]']             
                              (None, 512, 512, 1),        5                                       
                              (None, 512, 512, 1),                                                
                              (None, 256, 256, 1),                                                
                              (None, 256, 256, 1),                                                
                              (None, 128, 128, 1),                                          

In [ ]:
# from datetime import datetime 
# start1 = datetime.now() 

# train(d_model_fore, g_model_fore, gan_model_fore,d_model_back,g_model_back,gan_model_back,train_data_generator,mask_data_generator, n_epochs=300, n_batch=2) 


# stop1 = datetime.now()
# #Execution time of the model 
# execution_time = stop1-start1
# print("Execution time is: ", execution_time)